In [1]:
import sys
import os
import time
import datetime as dt
import math
import numpy as np 
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
from scipy.signal import butter,filtfilt
from scipy.interpolate import interp1d

In [2]:
# import activity analysis functions from aa.py file in the same folder as this jupiter notebook
%load_ext autoreload
%autoreload 1
%aimport aa

In [14]:
import scipy.io
mat = scipy.io.loadmat('data/stim4.mat')

In [193]:
def extract(tree):
    for x in range(len(tree)):
        if type(tree[x]) == list:
            tree[x] = extract(tree[x])
        else:
            return tree[x]

    return tree

In [29]:
len(mat['stim'][0][0]) #11


11

In [188]:
mat['stim'][0][0][0] # stimulus details
mat['stim'][0][0][1] # listening dates and times
mat['stim'][0][0][2] # time
mat['stim'][0][0][3] # corrugator sEMG
mat['stim'][0][0][4] # zygomaticus sEMG
mat['stim'][0][0][5] # trapezius sEMG
mat['stim'][0][0][6] # BVP
mat['stim'][0][0][7] # SCR
mat['stim'][0][0][8] # temp
mat['stim'][0][0][9] # Resp
mat['stim'][0][0][10] # Ratings
stim = mat['stim'][0][0]
arrayRef =[stim[1][0][0][0][0][1].dtype,stim[1][0][0][0][0][15].dtype]
print(stim[0])
print(stim[0][0][0][1][0])

Levels =(['track', 'sess', 'time', 'emgc', 'emgz', 'emgt', 'bvp', 'scon', 'temp', 'resp', 'beh'])
Level = [['piece', 'album', 'performer', 'composer', 'genre', 'stimN', 'duration']]
Level.append(['date', 'timeD', 'timeS', 'order', 'prev', 'notes', 'sneezes'])
Level.append(['s', 'sT', 'Hz'])
Level.append(['raw', 'filt', 'filtN', 'rms', 'rmsN'])
Level.append(['raw', 'filt', 'filtN', 'rms', 'rmsN'])
Level.append(['raw', 'filt', 'filtN', 'rms', 'rmsN'])
Level.append(['raw', 'filt', 'filtN', 'bpm', 'amp', 'beat'])
Level.append(['raw', 'filt', 'filtN', 'tonic', 'tonicN', 'ORt'])
Level.append(['raw', 'rawN', 'filt', 'filtN'])
Level.append(['raw', 'rawN', 'filt', 'filtN', 'bpm', 'phase', 'Inha'])
Level.append(['aro', 'val'])

LevelResponse = ['D', 'T', 'R', 'N']
Level

[[(array(['The Littlest Birds'], dtype='<U18'), array(['Blue Horse'], dtype='<U10'), array(['The Be Good Tanyas'], dtype='<U18'), array(['The Be Good Tanyas'], dtype='<U18'), array(['Folk'], dtype='<U4'), array([[4]], dtype=uint8), array([[246]], dtype=uint8))]]
Blue Horse


[['piece', 'album', 'performer', 'composer', 'genre', 'stimN', 'duration'],
 ['date', 'timeD', 'timeS', 'order', 'prev', 'notes', 'sneezes'],
 ['s', 'sT', 'Hz'],
 ['raw', 'filt', 'filtN', 'rms', 'rmsN'],
 ['raw', 'filt', 'filtN', 'rms', 'rmsN'],
 ['raw', 'filt', 'filtN', 'rms', 'rmsN'],
 ['raw', 'filt', 'filtN', 'bpm', 'amp', 'beat'],
 ['raw', 'filt', 'filtN', 'tonic', 'tonicN', 'ORt'],
 ['raw', 'rawN', 'filt', 'filtN'],
 ['raw', 'rawN', 'filt', 'filtN', 'bpm', 'phase', 'Inha'],
 ['aro', 'val']]

In [283]:
SRP ={}

md = 0
bit = {}
structNames = Level[md] 
for i in range(len(structNames)):
    det =stim[0][0][md][i].ravel()[0] #extract(stim[0][0][md][i])
    
    bit[structNames[i]] = det
SRP[Levels[md]]=bit

md +=1
bit = {}
structNames = Level[md] 
for i in range(len(structNames)):
    det =  stim[md][0][0][i].ravel()
    if len(det)==24:
        days = []
        for j in range(24):
            if str(det[j].dtype).startswith('<U'):
                if len(extract([det[j]]))>0:
                    days.append(extract(det[j]))
                else:
                    days.append([])
            else:
                if str(det[j].dtype).startswith('u'):
                    if det[j].shape == (0,0):
                        days.append('')
                    else:
                        days.append(det[j])
                    
                else:
                    days.append(det[j])
        det = days
    bit[structNames[i]] = det
SRP[Levels[md]]=bit    

md +=1
bit = {}
structNames = Level[md] 
for i in range(len(structNames)):
    det =stim[md][0][0][i].ravel()
    if len(det)==1:
        det = det[0]
    bit[structNames[i]] = det
SRP[Levels[md]]=bit


for md in range(3,11):
    bit = {}
    structNames = Level[md] 
    for i in range(len(structNames)):
        det =  stim[md][0][0][i].ravel()[0]
        if len(det)==4:
            signals = {LevelResponse[0]:pd.DataFrame(data=det[0],index=SRP['time']['s']),
                      LevelResponse[1]:det[1][0],
                      LevelResponse[2]:det[2][0][0],
                      LevelResponse[3]:det[3][0]}
            bit[structNames[i]] = signals
    SRP[Levels[md]]=bit  


In [285]:
SRP.keys()

dict_keys(['track', 'sess', 'time', 'emgc', 'emgz', 'emgt', 'bvp', 'scon', 'temp', 'resp', 'beh'])